## Module 1 Homework

In [54]:
import pandas_datareader as pdr
from datetime import date
import yfinance as yf

### Question 1: [Macro] Average growth of GDP in 2023

What is the average growth (in %) of GDP in 2023?; round to the nearest tenth digit.

In [2]:
# https://fred.stlouisfed.org/series/GDPC1
gdp_df = pdr.DataReader("GDPC1", "fred")

# YoY growth rate
gdp_df["GDP_GROWTH_YOY"] = gdp_df["GDPC1"] / gdp_df["GDPC1"].shift(4) - 1
gdp_df["GDP_GROWTH_YOY"] = gdp_df["GDP_GROWTH_YOY"] * 100

# In 2023, on average YoY growth rate for GDP
gdp_df[(gdp_df.index >= "2023-01-01") & (gdp_df.index <= "2023-10-01")]["GDP_GROWTH_YOY"].mean().round(1)

2.5

### Question 2: [Macro] Inverse "Treasury Yield"

Find the min value of the difference between *DGS10* and *DGS2* after since year 2000 (2000-01-01); round to the nearest tenth digit.

In [3]:
# https://fred.stlouisfed.org/series/DGS2
# https://fred.stlouisfed.org/series/DGS10
start = date(year=2000, month=1, day=1)
dgs2_df = pdr.DataReader("DGS2", "fred", start=start)
dgs10_df = pdr.DataReader("DGS10", "fred", start=start)

# Difference between DGS10 and DGS2 
inverse_yield_df = dgs10_df.join(dgs2_df)
inverse_yield_df["DIFF_DGS10_DGS2"] = inverse_yield_df["DGS10"] - inverse_yield_df["DGS2"]

# The lowest point of 10-year and 2-year yields
inverse_yield_df[inverse_yield_df["DIFF_DGS10_DGS2"] == inverse_yield_df["DIFF_DGS10_DGS2"].min()]

,DGS10,DGS2,DIFF_DGS10_DGS2
DATE,,,
2023-07-03,3.86,4.94,-1.08


**Inverted yield curve** happens when long-term treasury yield is lower than short-term treasury yield. This is an unusal event as normally the yield for the longer maturity asset should have more interest rate. 

The counter-intuitive situation often signifies the economic slowdown because it shows that investors are willing to accept lower yield in long-term bond (Higher risk in terms of time of investment)rather a higher yield in short-term bond (Lower risk in terms of time of investment).

### Question 3: [Index] Which Index is better recently?

Compare S&P 500 and IPC Mexico indexes by the 5 year growth and find which one has the highest growth by how much?

In [20]:
spx_df = yf.download("^GSPC", start="2019-04-09", end="2024-04-10")
ipc_df = yf.download("^MXX", start="2019-04-09", end="2024-04-10")

# 5Y growth
spx_5y_growth = (spx_df["Adj Close"].iloc[-1] - spx_df["Adj Close"].iloc[0]) / spx_df["Adj Close"].iloc[0] * 100
ipc_5y_growth = (ipc_df["Adj Close"].iloc[-1] - ipc_df["Adj Close"].iloc[0]) / ipc_df["Adj Close"].iloc[0] * 100

print(f"5Y growth on S&P500: {spx_5y_growth}")
print(f"5Y growth on IPC Mexico: {ipc_5y_growth}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

5Y growth on S&P500: 81.01279426847174
5Y growth on IPC Mexico: 27.50624912566744


In [28]:
# 10Y growth on SET
set_df = yf.download("^SET.BK", start="2014-04-09", end="2024-04-10")

(set_df["Adj Close"].iloc[-1] - set_df["Adj Close"].iloc[0]) / set_df["Adj Close"].iloc[0] * 100

[*********************100%%**********************]  1 of 1 completed


1.3813089210377276

In [47]:
# CAGR - Compound Annual Growth Rate; accounting for compounding investment over long period of time
n = (spx_df.index[-1] - spx_df.index[0]).days / 365.25
spx_cagr = ((spx_df["Adj Close"].iloc[-1] / spx_df["Adj Close"].iloc[0]) ** (1/n) - 1) * 100

n = (set_df.index[-1] - set_df.index[0]).days / 365.25
set_cagr = ((set_df["Adj Close"].iloc[-1] / set_df["Adj Close"].iloc[0]) ** (1/n) - 1) * 100

print(f"Annualized 5Y-CAGR on S&P500: {spx_cagr}")
print(f"Annualized 10Y-CAGR on SET: {set_cagr}")

Annualized 5Y-CAGR on S&P500: 12.595412447719557
Annualized 10Y-CAGR on SET: 0.1372609196827801


### Question 4: [Stocks OHLCV] 52-week range ratio (2023) for the selected stocks?

Find the largest range/spread ratio [=(max-min)/max] of Adj.Close prices in 2023

In [52]:
# Stock screener: top 6 earning stocks
# https://companiesmarketcap.com/most-profitable-companies/

tickers = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]
stock_df = yf.download(tickers, start="2023-01-01", end="2024-01-01")["Adj Close"]

[*********************100%%**********************]  6 of 6 completed


In [67]:
for ticker in tickers:
    range_ratio = (stock_df[stock_df.index == stock_df[ticker].idxmax()][ticker].values[0] - stock_df[stock_df.index == stock_df[ticker].idxmin()][ticker].values[0]) / stock_df[stock_df.index == stock_df[ticker].idxmax()][ticker].values[0]
    print(f"{ticker}: {range_ratio:.2f}")

2222.SR: 0.21
BRK-B: 0.21
AAPL: 0.37
MSFT: 0.42
GOOG: 0.39
JPM: 0.28


The largest range ratio indicates the high market volaitlity which is the opportunity for traders to make a profit.

### Question 5: [Stocks] Dividend Yield

Find the largest dividend yield for the same set of stocks

In [103]:
for ticker in tickers:
    dividends = yf.Ticker(ticker).dividends
    dividend_yield = dividends[(dividends.index >= "2023-01-01") & (dividends.index <= "2023-12-31")].sum() / stock_df[ticker].dropna().iloc[-1] * 100

    print(f"{ticker}: {dividend_yield}")

2222.SR: 2.7743470474138237
BRK-B: 0.0
AAPL: 0.4940592304162832
MSFT: 0.7433143940964608
GOOG: 0.0
JPM: 2.4096079718909573


### Question 6: [Exploratory] Investigate new metrics

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly)

Ans: As the tourist industry has a huge effect on Thailand's GDP, the growth of the number of tourist coming into the country seems to be a very interesting indicator for Thai stocks.

### Question 7: [Exploratory] Time-driven strategy description around earnings releases

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.

Ans: Using historical data on earning release dates, we can analyze and screen stocks that shows consistent patterns around their earnings releases. If a stock usually has an uptrend after its earning release, traders can use the momentum along with other technical indicators to find the best position to invest in.